# COVID-19 Status in Australia
> Tracking new, confirmed, and recovered cases and deaths by state.

- comments: true
- author: IamAshKS
- categories: [overview, interactive, australia]
- hide: true
- permalink: /covid-overview-australia/

In [1]:
#hide

from IPython.display import HTML
from pathlib import Path

import jinja2 as jj
import numpy as np
import pandas as pd
import requests as rq

In [2]:
#hide

def do_dev_tasks(html):
    file_url = '.local'
    
    if Path(file_url).is_dir():
        with open(f'{file_url}/index.html', 'w') as f:
            f.write(html)


def get_dataframe(name):
    data_url = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
               f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    
    return pd.read_csv(data_url)


def get_css_asset(name):
    data_url = f'https://raw.githubusercontent.com/iamashks/covid19-australia/master/assets/css/{name}.css'
    file_url = f'assets/css/{name}.css'
    
    if Path(file_url).is_file():
        asset = f'<link rel="stylesheet" type="text/css" href="../{file_url}" />\n'
    else:
        asset = f'<style>{rq.get(data_url).text}</style>'
    
    return asset


def get_template(name):
    data_url = f'https://raw.githubusercontent.com/iamashks/covid19-australia/master/templates/{name}.html'
    file_url = f'templates/{name}.html'
    
    if Path(file_url).is_file():
        templateLoader = jj.FileSystemLoader(searchpath='./')
        templateEnv = jj.Environment(loader=templateLoader)
        template = templateEnv.get_template(file_url)
    else:
        template = jj.Template(rq.get(data_url).text)
    
    return template

In [3]:
#hide

COL_COUNTRY = 'Country/Region'
COL_STATE = 'Province/State'
COUNTRY = 'Australia'

dft_confirm = get_dataframe('confirmed')
dft_confirm = dft_confirm[dft_confirm[COL_COUNTRY] == COUNTRY]
 
dft_demised = get_dataframe('deaths')
dft_demised = dft_demised[dft_demised[COL_COUNTRY] == COUNTRY]

dft_recover = get_dataframe('recovered')
dft_recover = dft_recover[dft_recover[COL_COUNTRY] == COUNTRY]

In [4]:
#hide

COL_TODAY = dft_confirm.columns[-1]
COL_1DAY = dft_confirm.columns[-1 - 1]
COL_5DAY = dft_confirm.columns[-1 - 5]
COL_50DAY = dft_confirm.columns[-1 - 50]

df_table = pd.DataFrame({'State': dft_confirm[COL_STATE], 'Cases': dft_confirm[COL_TODAY], 
                         'Recover': dft_recover[COL_TODAY], 'Deaths': dft_demised[COL_TODAY]})

df_table['Cases (5D)'] = np.array(dft_confirm[COL_TODAY]) - np.array(dft_confirm[COL_5DAY])
df_table['Recover (5D)'] = np.array(dft_recover[COL_TODAY]) - np.array(dft_recover[COL_5DAY])
df_table['Deaths (5D)'] = np.array(dft_demised[COL_TODAY]) - np.array(dft_demised[COL_5DAY])
df_table['Cases (1D)'] = np.array(dft_confirm[COL_TODAY]) - np.array(dft_confirm[COL_1DAY])
df_table['Recover (1D)'] = np.array(dft_recover[COL_TODAY]) - np.array(dft_recover[COL_1DAY])
df_table['Deaths (1D)'] = np.array(dft_demised[COL_TODAY]) - np.array(dft_demised[COL_1DAY])
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)

df_table = df_table.sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
df_table = df_table.reset_index()
df_table.index += 1

del df_table['index'] # del duplicate index
df_table.head(8)

,State,Cases,Recover,Deaths,Cases (5D),Recover (5D),Deaths (5D),Cases (1D),Recover (1D),Deaths (1D),Fatality Rate
1,New South Wales,3004,2227,34,33,301,8,2,19,0,1.1
2,Victoria,1349,1280,17,13,37,3,0,15,0,1.3
3,Queensland,1033,926,6,9,188,0,3,0,0,0.6
4,Western Australia,549,486,8,3,35,1,0,0,0,1.5
5,South Australia,438,414,4,0,26,0,0,3,0,0.9
6,Tasmania,214,132,11,9,55,4,2,9,0,5.1
7,Australian Capital Territory,106,100,3,2,7,0,0,1,0,2.8
8,Northern Territory,28,23,0,0,7,0,0,0,0,0.0


In [5]:
#hide

dt_cols = dft_confirm.columns[~dft_confirm.columns.isin([COL_STATE, COL_COUNTRY, 'Lat', 'Long'])]
dft_cases = dft_confirm.groupby(COL_STATE)[dt_cols].sum()
dft_cases_new = dft_cases.diff(axis=1).fillna(0).astype(int)

include_cols = ['Cases', 'Recover', 'Deaths', 'Cases (5D)', 'Recover (5D)', 'Deaths (5D)']

summary_nsw = df_table[df_table['State'].eq('New South Wales')][include_cols].sum().add_prefix('NSW ')
summary_vic = df_table[df_table['State'].eq('Victoria')][include_cols].sum().add_prefix('VIC ')
summary_qld = df_table[df_table['State'].eq('Queensland')][include_cols].sum().add_prefix('QLD ')

summary_time = {'updated': pd.to_datetime(COL_TODAY), 'since': pd.to_datetime(COL_5DAY)}
summary = {**summary_time, **df_table[include_cols].sum(), **summary_nsw, **summary_vic, **summary_qld}
summary

{'updated': Timestamp('2020-04-27 00:00:00'),
 'since': Timestamp('2020-04-22 00:00:00'),
 'Cases': 6721,
 'Recover': 5588,
 'Deaths': 83,
 'Cases (5D)': 69,
 'Recover (5D)': 656,
 'Deaths (5D)': 16,
 'NSW Cases': 3004,
 'NSW Recover': 2227,
 'NSW Deaths': 34,
 'NSW Cases (5D)': 33,
 'NSW Recover (5D)': 301,
 'NSW Deaths (5D)': 8,
 'VIC Cases': 1349,
 'VIC Recover': 1280,
 'VIC Deaths': 17,
 'VIC Cases (5D)': 13,
 'VIC Recover (5D)': 37,
 'VIC Deaths (5D)': 3,
 'QLD Cases': 1033,
 'QLD Recover': 926,
 'QLD Deaths': 6,
 'QLD Cases (5D)': 9,
 'QLD Recover (5D)': 188,
 'QLD Deaths (5D)': 0}

In [6]:
#hide_input

html_text = get_template('overview').render(D=summary, table=df_table, newcases=dft_cases_new, 
                                            np=np, pd=pd, enumerate=enumerate)

html_text = f'<div>{get_css_asset("keen")}{html_text}</div>'

do_dev_tasks(html=html_text)
HTML(html_text)

Visualizations by [Ashutosh KS (Ash)](https://ashks.com/)[^1][^2].

[^1]: Notebook: [Original](https://go.aksingh.net/covid-19-au) and [Source](https://github.com/iamashks/covid19-australia/)
[^2]: Data Source: [2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19)